## Building Your First Multi-Agent AI App with AutoGen

<a target="_blank" href="https://colab.research.google.com/github/designingwithml/blogposts/blob/main/notebooks/autogen_introduction.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Multi-agent AI apps (a setup where multiple agents powered by GenAI models collaborate autonomously) hold promise in addressing complex tasks. In this notebook, the goal is to explore how we might build an agent workflow that can address multiple tasks.  

## LLM Configuration Specification 

AutoGen agents take an `llm_config` argument that specifies the LLM configuration for an Agent. 
Importantly, AutoGen standardizes on the openai api format for specifying an LLM. This means the openai and azure openai api are supported out of the box. For Azure Open AI models, the `base_url` and `api_version`  and `api_type` fields are required.

- model: The name of the model 
- api_key: The API key to use.
- base_url: The base URL of the LLM api 
- api_version:  The version of the LLM api (Azure Open AI only)
- api_type: The type of LLM api (Azure Open AI only)


In [1]:
import os
llm_config = {"config_list": [{
        "model": "gpt-4-turbo-preview",
        "api_key": os.environ["OPENAI_API_KEY"],  # Note, add your own API key here
    }],
    "temperature": 0.9, 
    "cache_seed": 41,
}

## Defining a Basic Agent Workflow with Two Agents 

In the following section we will define an agent workflow with only two agents. So what is the orchestation behavior i.e. how the agents interact or decide when to act? Well, this is fairly simple as they will reply to each others messages until the task is completed.

In [2]:
from autogen import AssistantAgent, UserProxyAgent 

assistant = AssistantAgent(
    name= "assistant", llm_config=llm_config)
user_proxy = UserProxyAgent(
    name = "user_proxy", 
    code_execution_config={"work_dir": 
                           "data/scratch", 
                           "use_docker": False},
    human_input_mode="NEVER",
    is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("TERMINATE") if x.get("content") else False 
    )  

chatresults = user_proxy.initiate_chat(assistant, message="Plot a chart of NVDA and TESLA stock price change YTD. Save the result to a file named chart.png.") 

user_proxy (to assistant):

Plot a chart of NVDA and TESLA stock price change YTD. Save the result to a file named chart.png.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To plot a chart of NVIDIA (NVDA) and Tesla (TSLA) stock price changes year-to-date (YTD), we will follow these steps:

1. **Collect stock data:** Use an API or a Python library that provides access to stock market data. For this purpose, `yfinance` is a popular choice as it allows us to fetch historical market data from Yahoo Finance.
2. **Extract YTD data:** Filter the data to only include information from the start of the current year to the present date.
3. **Plot the chart:** Use a plotting library, such as `matplotlib`, to create a chart visualizing the stock price changes of NVDA and TSLA YTD.
4. **Save the chart:** Save the resulting plot to a file named `chart.png`.

Before we proceed, ensure that `yfinance` and `matplotlib` are installed in your

/opt/homebrew/Caskroom/miniconda/base/envs/autog/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



>>>>>>>> EXECUTING CODE BLOCK 1 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 



--------------------------------------------------------------------------------
assistant (to user_proxy):

The necessary libraries (`yfinance` and `matplotlib`) are successfully installed in your environment, and any required dependencies have been updated or confirmed to be present. 

At this stage, you should execute the provided Python script to plot the YTD stock price changes for NVDA and TSLA and save it to `chart.png`. Since it seems you have already run the command to install the necessary packages, you can now proceed to run the script that generates the chart.

Please ensure to run the previously provided Python script named `plot_stock_prices.py` in your environment. This script will fetch the year-to-date stock prices for NVIDIA and Tesla, plot them, and save the chart as `chart.png` in the current directory.

If the script has b

## Tool Calling 

In this section we will review the importance of tools. Consider a scenario where our agent workflow needs to generate images. Let's see how they fare. 

In [5]:
user_proxy.initiate_chat(assistant, message="paint a beautiful picture of a cat and a dog playing together.")

user_proxy (to assistant):

paint a beautiful picture of a cat and a dog playing together.

--------------------------------------------------------------------------------
assistant (to user_proxy):

I can help you create a simple "painting" of a cat and a dog playing together using Python code with ASCII art. This won't be a traditional painting but a creative representation using characters. Let's do it step by step.

First, we will create a simple ASCII art of a cat and a dog playing together. Bear in mind that ASCII art is quite limited in detail, so "beautiful" is subjective and this will be a basic representation. 

```python
# filename: cat_and_dog_playing.py

# Cat and Dog Playing Together in ASCII Art

print("""
    /\_/\  
   ( o.o ) > What a beautiful day to play!
    > ^ <                           /    
         ______                    /     
    _/|_||_||_\__               \|/      
   /___n_n___\                  (o o)    
    /       \__                -(")-    
   /

ChatResult(chat_id=None, chat_history=[{'content': 'paint a beautiful picture of a cat and a dog playing together.', 'role': 'assistant'}, {'content': 'I can help you create a simple "painting" of a cat and a dog playing together using Python code with ASCII art. This won\'t be a traditional painting but a creative representation using characters. Let\'s do it step by step.\n\nFirst, we will create a simple ASCII art of a cat and a dog playing together. Bear in mind that ASCII art is quite limited in detail, so "beautiful" is subjective and this will be a basic representation. \n\n```python\n# filename: cat_and_dog_playing.py\n\n# Cat and Dog Playing Together in ASCII Art\n\nprint("""\n    /\\_/\\  \n   ( o.o ) > What a beautiful day to play!\n    > ^ <                           /    \n         ______                    /     \n    _/|_||_||_\\__               \\|/      \n   /___n_n___\\                  (o o)    \n    /       \\__                -(")-    \n   /| |     || |\\          

Above, the agents do not fare particularly well. They take a simple approach and generate ascii art images. Depending on how this specific task is framed, the agents could explore even more problemative approaches - e.g., attempting to install random image generation libraries, assuming the availability of a GPU, might hallucinate methods in libraries, encounter multiple errors, retries etc. All very inefficient.

In cases like the above, where we have a specific approach to solving the task, we can provide the agents with tools that encode our approach.

In [6]:
from typing import List
import uuid
import requests
from pathlib import Path
from openai import OpenAI

def generate_and_save_images(query: str, image_size: str = "1024x1024") -> List[str]:
    client = OpenAI()
    response = client.images.generate(model="dall-e-3", prompt=query, n=1, size=image_size)
    saved_files = []
    if response.data:
        for image_data in response.data:
            file_name = str(uuid.uuid4()) + ".png" 
            file_path = Path("data") / file_name
            img_url = image_data.url
            img_response = requests.get(img_url)
            if img_response.status_code == 200:
                with open(file_path, "wb") as img_file:
                    img_file.write(img_response.content)
                    saved_files.append(str(file_path))
                    print(f"Image generated and saved to {file_path}")
    return saved_files 

In [7]:

from autogen import register_function
register_function(
    generate_and_save_images,
    caller=assistant,  # The assistant agent can suggest calls to the  function
    executor=user_proxy,  # The user proxy agent can execute the function call.
    name="generate_and_save_images",  # By default, the function name is used as the tool name.
    description="Generate images using DALL-E",
)

In [9]:
user_proxy.initiate_chat(assistant, message="paint a beautiful picture of a cat and a dog playing together.")

user_proxy (to assistant):

paint a beautiful picture of a cat and a dog playing together.

--------------------------------------------------------------------------------


assistant (to user_proxy):

***** Suggested tool call (call_bSDUgG9khJB15diBTZDsKdDe): generate_and_save_images *****
Arguments: 
{"query":"a cat and a dog playing together, beautiful"}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION generate_and_save_images...
Image generated and saved to data/8fcf212f-7119-48ca-b0b8-302932fdf3fb.png
user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool (call_bSDUgG9khJB15diBTZDsKdDe) *****
["data/8fcf212f-7119-48ca-b0b8-302932fdf3fb.png"]
**********************************************************************

--------------------------------------------------------------------------------
assistant (to user_proxy):

I've created a beautiful picture of a cat and a dog playing together. You can view the picture by following the link below:

[View Picture](sandbox:/dat

ChatResult(chat_id=None, chat_history=[{'content': 'paint a beautiful picture of a cat and a dog playing together.', 'role': 'assistant'}, {'tool_calls': [{'id': 'call_bSDUgG9khJB15diBTZDsKdDe', 'function': {'arguments': '{"query":"a cat and a dog playing together, beautiful"}', 'name': 'generate_and_save_images'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': '["data/8fcf212f-7119-48ca-b0b8-302932fdf3fb.png"]', 'tool_responses': [{'tool_call_id': 'call_bSDUgG9khJB15diBTZDsKdDe', 'role': 'tool', 'content': '["data/8fcf212f-7119-48ca-b0b8-302932fdf3fb.png"]'}], 'role': 'tool'}, {'content': "I've created a beautiful picture of a cat and a dog playing together. You can view the picture by following the link below:\n\n[View Picture](sandbox:/data/8fcf212f-7119-48ca-b0b8-302932fdf3fb.png)\n\nTERMINATE", 'role': 'user'}], summary="I've created a beautiful picture of a cat and a dog playing together. You can view the picture by following the link below:\n\n[View Pic